# Advanced OOP
Here, I follow the LinkedIn Learning course [Advanced Python: Object-Oriented
Programming](https://www.linkedin.com/learning/advanced-python-object-oriented-programming/advanced-object-oriented-programming-oop?resume=false&u=72605090)
by Miki Tebeka and try the code. Here is the [course
repo](https://github.com/LinkedInLearning/advanced-python-object-oriented-programming-4510177)
(it is very good). I have extensively added additional explanations that I came across
during researching.

## Part 4: Metaclasses

### What are metaclasses

- course describes them as "class of classes"
    - I do remember it was a way to re-define a classes' base class, though (?)

- `ABCMeta` from the abc module is used for defining ABCs and a helper class ABC to alternatively define ABCs through inheritance

- `type` is the metaclass for most classes
    - which is why we can check if something is a class by doing `if type(obs) is type`

In [8]:
# first demonstration

class Robot:
    manufacture = 'BnL'

    def move(self, x, y):
        print(f'{self} moving to {x}/{y}')


walle = Robot()
walle.move(100, 200)

print(type(Robot))
print(Robot.__bases__)


<__main__.Robot object at 0x73d9a9c7ea20> moving to 100/200
<class 'type'>
(<class 'object'>,)


--> `type(Robot)` returns `type`, because all classes are instances of type

--> `Robot.__bases__` returns `object`, because all new-style classes inherit from `object` 

In Python, everything is an object, and every class itself is an instance of a metaclass
`type`.

If I create a new class, the `type()` build-in function is called. <br>
This function, [called with three arguments, returns a new type object](https://docs.python.org/3/library/functions.html#type).

These are practice examples from scikit-learn to use `type()` to create a new class:
- [`type("SubEstimator", (), {"attribute_present": True})`](https://github.com/scikit-learn/scikit-learn/blob/031d2f83b7c9d1027d1477abb2bf34652621d603/sklearn/utils/tests/test_validation.py#L1205)
- [`estimator = type(self.estimator)(criterion=self.criterion)`](https://github.com/scikit-learn/scikit-learn/blob/031d2f83b7c9d1027d1477abb2bf34652621d603/sklearn/ensemble/_forest.py#L372)
    - similar: [`return type(self)(self._x, idx)`](https://github.com/data-apis/array-api-extra/blob/dbcd7097f13afff0e5e00077b6050c387cc4c30a/src/array_api_extra/_lib/_at.py#L222) (in array_api_extra)
    - also similar: [`return type(meta_estimator)(estimator, param_grid, **extra_params)`](https://github.com/scikit-learn/scikit-learn/blob/031d2f83b7c9d1027d1477abb2bf34652621d603/sklearn/tests/test_metaestimators.py#L218C13-L218C79) (in scikit-learn)

In [ ]:
# What `class` keyword does
from textwrap import dedent # dedent removes one level of identation

class_body = '''
    manufacture = 'BnL'

    def move(self, x, y):
        print(f'{self} moving to {x}/{y}')
'''

cls_dict = {}
exec(dedent(class_body), None, cls_dict) # creating a dict from a string
print(cls_dict) # holds both attributes: the class attribute `manufacture` and the method `move`
# {'manufacture': 'BnL', 'move': <function move at 0x73d9a9c9dc60>}

move = cls_dict['move']
print(move.__code__.co_varnames)
# ('self', 'x', 'y')

move(walle, 10, 20) # as before
# <__main__.Robot object at 0x73d9a9c7ea20> moving to 10/20

{'manufacture': 'BnL', 'move': <function move at 0x73d9a9c9dc60>}
('self', 'x', 'y')
<__main__.Robot object at 0x73d9a9c7ea20> moving to 10/20


In [ ]:
# Using `type(`) to create a new class
Robot = type(
    'Robot',
    (object,),
    cls_dict, # we could also just pass a dict rather than creating one from a string
)
walle = Robot()
walle.move(100, 200)

<__main__.Robot object at 0x73d9a9c94e30> moving to 100/200


[Python's classobject.c file](https://github.com/python/cpython/blob/main/Objects/classobject.c) holds the code use to create classes.
    
- (I cannot read that file yet, but it looks remarkable readable if you know some `C`:
  short and simple.)

### Using metaclasses to intercept class creation

- `__new__` in a metaclass is called before the class is created; it's responsible for
  returning the new class object

- `__init__` in a metaclass is called after the class is created; it's used to customize
  or initialize the class object (not the instance of the class object!!!)

- if we overwrite both, we can create a new metaclass (that differs from `type`)

Following example shows how to intercept class creation by injecting a new attribute
`created` into the class:

In [ ]:
from datetime import datetime

class CheckerMeta(type):
    """
    If you have a procedure with 10 parameters, you probably missed some.
        - Alan J. Perlis
    """
    def __new__(mclass, name, bases, mapping):
        # `name` is the name of the class being created as a str
        # `bases` is a tuple of base classes our new class should inherit from
        # `mapping` is a dict on the class body
        # these are exactly the same as in the `type()` function
        print(f'[checker] Creating class {name} with {bases}')
        mapping['created'] = datetime.now() # adds a timestamp attribute to every class created 
        return type.__new__(mclass, name, bases, mapping)

    def __init__(cls, name, bases, mapping):
        count = sum(1 for v in mapping.values() if callable(v))
        if count > 10:
            raise ValueError(f'{name} has too many methods ({count})')
        print(f'[checker] {name} with {count} methods')
        return type.__init__(cls, name, bases, mapping)


class Checker(metaclass=CheckerMeta):
    pass

print(type(Checker)) # it is `CheckerMeta`, not `type`
print('')

class Robot(Checker):
    manufacture = 'BnL'

    def move(self, x, y):
        print(f'{self} moving to {x}/{y}')


walle = Robot()
walle.move(100, 200)

print(type(Robot))
print(Robot.created) 

[checker] Creating class Checker with ()
[checker] Checker with 0 methods
<class '__main__.CheckerMeta'>

[checker] Creating class Robot with (<class '__main__.Checker'>,)
[checker] Robot with 1 methods
<__main__.Robot object at 0x73d9a9c97ec0> moving to 100/200
<class '__main__.CheckerMeta'>
2025-06-17 11:13:07.828079


### Using metaclasses to intercept instance creation

- `__new__` and `__init__` control the behavior of class creation of subclasses, not the
  instance creation, which we override `__call__` for:

In [ ]:
class CheckerMeta(type):
    """
    If you have a procedure with 10 parameters, you probably missed some.
        - Alan J. Perlis
    """
    def __new__(mclass, name, bases, mapping):
        print(f'[checker] Creating class {name} with {bases}')
        mapping['created'] = datetime.now()
        return type.__new__(mclass, name, bases, mapping)

    def __init__(cls, name, bases, mapping):
        count = sum(1 for v in mapping.values() if callable(v))
        if count > 10:
            raise ValueError(f'{name} has too many methods ({count})')
        print(f'[checker] {name} with {count} methods')
        return type.__init__(cls, name, bases, mapping)

    def __call__(cls, *args, **kw):
        # we are overriding __call__, which is used to instantiate the instances
        print(f'[checker] instance of {cls.__name__} created')
        if (count := len(args) + len(kw) > 10):
            name = cls.__name__
            raise ValueError(f'{name} instance with too many arguments ({count})')
        return type.__call__(cls, *args, **kw) # create the instance with build-in `type()` function


class Checker(metaclass=CheckerMeta):
    pass


print('')

class Robot(Checker):
    manufacture = 'BnL'

    def move(self, x, y):
        print(f'{self} moving to {x}/{y}')


walle = Robot() # `__call__` is now used to create the instance
walle.move(100, 200)

[checker] Creating class Checker with ()
[checker] Checker with 0 methods

[checker] Creating class Robot with (<class '__main__.Checker'>,)
[checker] Robot with 1 methods
[checker] instance of Robot created
<__main__.Robot object at 0x73d9a9cdd550> moving to 100/200


### Using metaclasses for attribute lookup

In [18]:
class CheckerMeta(type):
    """
    If you have a procedure with 10 parameters, you probably missed some.
        - Alan J. Perlis
    """
    def __new__(mclass, name, bases, mapping):
        print(f'[checker] Creating class {name} with {bases}')
        mapping['created'] = datetime.now()
        return type.__new__(mclass, name, bases, mapping)

    def __init__(cls, name, bases, mapping):
        count = sum(1 for v in mapping.values() if callable(v))
        if count > 10:
            raise ValueError(f'{name} has too many methods ({count})')
        print(f'[checker] {name} with {count} methods')
        return type.__init__(cls, name, bases, mapping)

    def __call__(cls, *args, **kw):
        print(f'[checker] instance of {cls.__name__} created')
        if (count := len(args) + len(kw) > 10):
            name = cls.__name__
            raise ValueError(f'{name} instance with too many arguments ({count})')
        return type.__call__(cls, *args, **kw)

    def __setattr__(cls, attr, value):
        old = getattr(cls, attr, None)
        name = cls.__name__
        print(f'[checker] {name}:{attr} {old!r} -> {value!r}')
        if value is None:
            raise ValueError(f'{name} sets {attr} to None')
        type.__setattr__(cls, attr, value)


class Checker(metaclass=CheckerMeta):
    pass


class Robot(Checker):
    manufacture = 'BnL'

    def move(self, x, y):
        print(f'{self} moving to {x}/{y}')


walle = Robot()
walle.move(100, 200)
Robot.manufacture = 'Boston Dynamics'

[checker] Creating class Checker with ()
[checker] Checker with 0 methods
[checker] Creating class Robot with (<class '__main__.Checker'>,)
[checker] Robot with 1 methods
[checker] instance of Robot created
<__main__.Robot object at 0x73d9a8ba2d80> moving to 100/200
[checker] Robot:manufacture 'BnL' -> 'Boston Dynamics'


Exercise: Create a Singleton (class that can only be instantiated once).

In [ ]:
class Singleton(type):
    subclass_created = False

    def __new__(mclass, name, bases, mapping):
        return type.__new__(mclass, name, bases, mapping)

    def __init__(cls, name, bases, mapping):
        return type.__init__(cls, name, bases, mapping)

    def __call__(cls):
        print(Singleton.subclass_created)
        if not Singleton.subclass_created:
            Singleton.subclass_created = True
            Singleton.instance = type.__call__(cls)
            return Singleton.instance # create new instance
        else:
            return Singleton.instance # return the already existing instance
    

class Driver(metaclass=Singleton):
    pass

d1 = Driver()
print(d1)

d2 = Driver()
print(d2)

print(d1 is d2)

False
True
True


In [ ]:
# chat-GPT suggested some improvements:

class Singleton(type):

    # note we don't need `__new__` since type class already has it and we don't need to
    # override it here

    def __init__(cls, name, bases, mapping):
        # using `cls` avoids hardcoding the metaclass name and makes it easier to work
        # with inheritance
        cls._instance = None
        super().__init__(name, bases, mapping) # more common

    def __call__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__call__(*args, **kwargs)
        return cls._instance
    

class Driver(metaclass=Singleton):
    pass

d1 = Driver()
print(d1)

d2 = Driver()
print(d2)

print(d1 is d2)

True


In [39]:
# solution from the course:
class SingletonMeta(type):
    def __call__(cls, *args, **kw):
        inst = getattr(cls, '_instance', None)
        if inst is None:
            inst = cls._instance = type.__call__(cls, *args, **kw)
        return inst


class Singleton(metaclass=SingletonMeta):
    pass


class Driver(Singleton):
    pass

d1 = Driver()
d2 = Driver()
print(d1 is d2)

True
